In [ ]:
from __future__ import print_function
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import dalmatian as dm
import pandas as pd

In [ ]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '1uqCOos-T9EMQU7y2ZUw4Nm84opU5fIT1y7jet1vnScE'
SAMPLE_RANGE_NAME = 'Omics!A:L'

creds = None

namespace="broad-firecloud-ccle"
workspace="DepMap_WES_CN_HG38"

In [ ]:
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            '/Users/jeremie/Documents/Projects/amcredentials.json', SCOPES)
        creds = flow.run_local_server()
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                            range=SAMPLE_RANGE_NAME).execute()
values = result.get('values', [])

In [ ]:
wm = dm.WorkspaceManager(namespace, workspace)

In [ ]:
samples = wm.get_samples()
ids = samples[samples['Source']!="sanger"].index.tolist()
ids = [val[val.index('ACH'):] for val in ids]

In [ ]:
len(toload)

In [ ]:
values = pd.DataFrame(values[1:],columns=values[0])

In [ ]:
values

In [ ]:
toload = set(values['DepMap ID'].tolist())-set(ids)
toload

In [ ]:
toload = values[values['DepMap ID'].isin(toload)][values['WES Owner']!='Sanger'][values['WES Status']=='Complete']["WES Data Links"].tolist()

In [ ]:
toload

In [ ]:
values[values["WES Data Links"].isin(list(toload))]

In [ ]:
with open('urls.txt', 'w') as f:
    for item in toload:
        f.write("%s\n" % item)

In [ ]:
! cat urls.txt | parallel -j 10 'curl -L -s {} | gsutil cp - "gs://fc-c7d47ebf-26a0-404c-8ef6-a1eb59c32e54/"$(echo {} | cut -d \/ -f 7)'